<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/11_cnn_pytorch/08_segnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SegNet

---

## 目的
セマンティックセグメンテーションとは何か理解する．\
SegNetの構造を理解する．\
SegNetを用いてARCDatsetでセグメンテーションを行う．

## セマンティックセグメンテーション

セマンティックセグメンテーションは,
画像内のオブジェクトをピクセル単位でクラス分類を行うタスクです．\
複数の物体を認識することができ，物体位置や形状も認識することができます．
自動運転や医療画像などの分野で使用されています．

<img src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/377968/d63667de-412a-3b38-afb4-6af4e6dae645.png" width = 30%>


<img src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/377968/3ffffea2-2a68-4fe4-1c81-b19748ecc2ff.png" width = 30%>


## SegNet

セマンティックセグメンテーションのネットワークは全結合層が無くなり．すべての層が畳み込み層となっています．\
最後の畳み込み層では,元画像と同じサイズ確率マップをクラス数分出力します．\
その確率マップにソフトマックス関数を使い最終的な出力結果とします．
<img src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/377968/94b5a80e-b1aa-87e4-4229-270c4fd440d7.png" width = 70%>

### エンコーダデコーダ構造
エンコーダでは，入力画像に対して畳み込みとpoolingを繰り返すことで，圧縮していき特徴マップを抽出する役割をしています．\
デコーダでは，unpoolingと畳み込みを繰り返して，圧縮した特徴マップを元のサイズに戻していきます． エンコーダデコーダ構造にすることで，省メモリ化といった効果があります．


## Pooling indices
プーリング処理は，繰り返しおこなうことで局所的な特徴が欠落してしまいます．
これでは，オブジェクトの境界部分などが曖昧になってしまいます．
 そこでSegNetでは，エンコーダでMaxPoolingを行ったときに最大値の位置情報を記録します，そして，unpoolingするときにその位置情報を使ってピクセルを戻していきます．この時，記録されていない位置には0が入ります．\
<img src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/377968/c1c96d67-7a1a-8978-f279-b5706e867dce.png" width = 70%>


必要なモジュールのインポート

In [ ]:
import os
from PIL import Image, ImageOps, ImageFilter
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms.functional import normalize
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import sys
import glob
import numbers
import random
import matplotlib.pyplot as plt
import torchsummary
import cv2
import tqdm

GPUの確認

In [ ]:
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

GPUの確認です．

In [ ]:
!nvidia-smi

# データセットの用意
今回は，画像サイズは半分，枚数を削減したARCDatsetを使用します．\
40のオブジェクトと背景の全41クラスがラベル付けされています．\
各クラスのラベル付けは以下のように定義されています．
<img src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/377968/5c634cce-1c98-48e7-ae16-c2ea3475d668.png" width = 70%>



In [ ]:
!wget http://www.mprg.cs.chubu.ac.jp/~masaki/share/ARCdataset_png2.zip
!unzip ./ARCdataset_png2.zip

## データセットクラス

セマンティックセグメンテーションは入力画像とラベル画像を用意する必要があります\
そのため，専用のDatasetクラスやデータ拡張クラスを用意しなければなりません．\
専用のDatasetクラスを用意します．\
Datsetクラスを作成するときは，torch.utils.data.Datasetを継承してオーバーライドします．



In [ ]:
def is_image(filename):
    return any(filename.endswith(ext) for ext in '.png')

def is_label(filename):
    return filename.endswith("_s.png")

def image_basename(filename):
    return os.path.basename(os.path.splitext(filename)[0])

class MYDataset(Dataset):
    
    def __init__(self, split, transform):
        self._base_dir = './ARCdataset_png/'
        
        self.split = split
        self.images_root = os.path.join(self._base_dir, split, 'rgb/')
        self.labels_root = os.path.join(self._base_dir, split, 'label/')
        
        self.filenames = [image_basename(f)
            for f in os.listdir(self.images_root) if is_image(f)]
        self.filenames.sort()

        self.filenamesGt = [image_basename(f)
            for f in os.listdir(self.labels_root) if is_label(f)]
        self.filenamesGt.sort()

        self.transform = transform
    
    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
         # 1. 画像読み込み
        image_file_path = self.filenames[index]+ '.png'
        image_file_path = os.path.join(self._base_dir, self.split, 'rgb/', image_file_path)
        img = Image.open(image_file_path).convert('RGB')

        # 2. アノテーション画像読み込み
        label_file_path = self.filenamesGt[index]+ '.png'
        label_file_path = os.path.join(self._base_dir, self.split, 'label/', label_file_path)
        label_class_img = Image.open(label_file_path).convert('L')      
        sample = {'image': img, 'label': label_class_img}

        # 3. データ拡張を実施
        return self.transform(sample)

## データ拡張
セマンティックセグメンテーションでデータ拡張する場合は，入力画像とラベル画像に同じ処理を行なう必要があります．\
Pytorchのデータ拡張は，入力画像とラベル画像を同時に行えないため専用のものを用意します．\
今回は切り抜き，正規化の処理をPILという画像処理ライブラリを使い実装します．

In [ ]:
class Normalize(object):
    def __init__(self, mean=(0., 0., 0.), std=(1., 1., 1.)):
        self.mean = mean
        self.std = std

    def __call__(self, sample):
        img = sample['image']
        mask = sample['label']
        img = np.array(img).astype(np.float32)
        mask = np.array(mask).astype(np.float32)
        img /= 255.0
        img -= self.mean
        img /= self.std
       
        return {'image': img,
                'label': mask}

class ToTensor(object):
    def __call__(self, sample):
       
        img = sample['image']
        mask = sample['label']
        img = np.array(img).astype(np.float32).transpose((2, 0, 1))
        mask = np.array(mask).astype(np.float32)

        img = torch.from_numpy(img).float()
        mask = torch.from_numpy(mask).float()

        return {'image': img,
                'label': mask}



class RandomCrop(object):
    def __init__(self, size, padding=0):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size # h, w
        self.padding = padding

    def __call__(self, sample):
        img, mask = sample['image'], sample['label']

        if self.padding > 0:
            img = ImageOps.expand(img, border=self.padding, fill=0)
            mask = ImageOps.expand(mask, border=self.padding, fill=255)

        assert img.size == mask.size
        w, h = img.size
        th, tw = self.size # target size
        x1 = random.randint(0, w - tw)
        y1 = random.randint(0, h - th)
        img = img.crop((x1, y1, x1 + tw, y1 + th))
        mask = mask.crop((x1, y1, x1 + tw, y1 + th))

        return {'image': img,
                'label': mask}


## データローダの作成
`transforms.Compose`を使い，使用するデータ拡張を設定します．先ほど作成したクロップ，画像の正規化のクラスを使用します．\
次にMYDatasetクラスには学習データか検証データどちらを使用するかとデータ拡張の設定を与えます．\
作成したMYDatasetクラスは`DataLoader`に与えます．
ミニバッチは6とします．


In [ ]:
# データ拡張を設定
transform = transforms.Compose([                          
    RandomCrop((320,320)), 
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ToTensor(),
])

test_transform = transforms.Compose([
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ToTensor(),
])

# データセットの作成
train_dataset = MYDataset(split='train', transform=transform)
val_dataset = MYDataset(split='val', transform=test_transform)

# データローダーの作成
batch_size = 5
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=1, shuffle=True,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=1, num_workers=0, shuffle=False,pin_memory=True)



# ネットワークモデルの定義
SegNetを定義します．

エンコーダでは，畳み込み層，BatchNorm，ReLUを2，3回繰り返した後MaxPoolingを行います．Pooling後はチャンネル数を増やして最終的に512チャンネルになります．
Pooling処理は，`F.max_pool2d()`の引数を`return_indices=True`にすることで最大値を取った場所の位置情報を獲得することができます．

また，各Pooling前の特徴マップのサイズ情報も獲得しておきます


デコーダでは，まずアンプ―リング処理を行います．
その後，逆畳み込み処理，BatchNorm，ReLUを2，3回繰り返していきます．
エンコーダでは，チャンネル数を徐々に増やしていきましたが，デコーダではチャンネル数を徐々に減らしていきます．
unpooling処理では，位置情報と出力する得著マップのサイズを渡します.\
`F.max_unpool2d()`の二つ目の引数に`F.max_pool2d()`で獲得した位置情報を与え,
`output_siz`に出力する特徴マップサイズを与えます．

デコーダの最終層はクラス数のチャンネルを出力します．


In [ ]:
class SegNet(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(SegNet, self).__init__()

        # Encoder layers

        self.encoder_0 = nn.Sequential(nn.Conv2d(in_channels=input_channels, out_channels=64, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(64),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(64),
                                            nn.ReLU(inplace=True))

        self.encoder_1= nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(128),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(128),
                                            nn.ReLU(inplace=True))

        self.encoder_2 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(256),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(256),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(256),
                                            nn.ReLU(inplace=True))

        self.encoder_3 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(512),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(512),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=512,out_channels=512, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(512),
                                            nn.ReLU(inplace=True))

        self.encoder_4 = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(512),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(512),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(512),
                                            nn.ReLU(inplace=True))

        # Decoder layers

        self.decoder_4 = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(512),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(512),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(512),
                                            nn.ReLU(inplace=True))

        self.decoder_3 = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(512),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(512),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(256),
                                            nn.ReLU(inplace=True))

        self.decoder_2 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(256),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(256),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(128),
                                            nn.ReLU(inplace=True))

        self.decoder_1 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(128),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(64),
                                            nn.ReLU(inplace=True))

        self.decoder_0 = nn.Sequential(nn.Conv2d(in_channels=64,out_channels=64, kernel_size=3, padding=1),
                                            nn.BatchNorm2d(64),
                                            nn.ReLU(inplace=True),
                                            nn.Conv2d(in_channels=64, out_channels=output_channels, kernel_size=1))

        self._init_weight()

    def forward(self, x):
        """
        Forward pass `input_img` through the network
        """

        # Encoder

        # Encoder Stage - 1
        dim_0 = x.size()
        x = self.encoder_0(x)
        x, indices_0 = F.max_pool2d(x, kernel_size=2, stride=2, return_indices=True)

        # Encoder Stage - 2
        dim_1 = x.size()
        x = self.encoder_1(x)
        x, indices_1 = F.max_pool2d(x, kernel_size=2, stride=2, return_indices=True)

        # Encoder Stage - 3
        dim_2 = x.size()
        x = self.encoder_2(x)
        x, indices_2 = F.max_pool2d(x, kernel_size=2, stride=2, return_indices=True)

        # Encoder Stage - 4
        dim_3 = x.size()
        x = self.encoder_3(x)
        x, indices_3 = F.max_pool2d(x, kernel_size=2, stride=2, return_indices=True)

        # Encoder Stage - 5
        dim_4 = x.size()
        x = self.encoder_4(x)
        x, indices_4 = F.max_pool2d(x, kernel_size=2, stride=2, return_indices=True)

        # Decoder

        #dim_d = x.size()

        # Decoder Stage - 5
        x = F.max_unpool2d(x, indices_4, kernel_size=2, stride=2, output_size=dim_4)
        x = self.decoder_4(x)
        #dim_4d = x.size()

        # Decoder Stage - 4
        x = F.max_unpool2d(x, indices_3, kernel_size=2, stride=2, output_size=dim_3)
        x = self.decoder_3(x)
        #dim_3d = x.size()

        # Decoder Stage - 3
        x = F.max_unpool2d(x, indices_2, kernel_size=2, stride=2, output_size=dim_2)
        x = self.decoder_2(x)
        #dim_2d = x.size()

        # Decoder Stage - 2
        x = F.max_unpool2d(x, indices_1, kernel_size=2, stride=2, output_size=dim_1)
        x = self.decoder_1(x)

        # Decoder Stage - 1
        x = F.max_unpool2d(x, indices_0, kernel_size=2, stride=2, output_size=dim_0)
        x = self.decoder_0(x)
        
        return x

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

# 学習

## ネットワークの設定

定義したネットワークを作成します．
`SegNet`クラスを呼び出して，ネットワークモデルを定義します．
また，GPUを使う場合（`use_cuda == True`）には，ネットワークモデルをGPUメモリ上に配置します．
これにより，GPUを用いた演算が可能となります．

学習を行う際の最適化方法としてモーメンタムSGD(モーメンタム付き確率的勾配降下法）を利用します．
また，学習率を0.01，モーメンタムを0.9として引数に与えます．

定義したネットワーク情報を`torchsummary.summary()`関数を用いて表示ます．

In [ ]:
import time
num_class = 41
model = SegNet(input_channels=3, output_channels=num_class)
if use_cuda:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

#エポック数の設定
epoch_num = 10

# 誤差関数の設定
criterion = nn.CrossEntropyLoss(reduction='mean')
if use_cuda:
    criterion.cuda()

#モデルの情報を表示
torchsummary.summary(model,(3,128,128))

## 評価関数の設定
mIoUとAccuracyの設定を行います．

IoUはIntersection(領域の共通部分) over Union(領域の和集合)の略で，セマンティックセグメンテーションでよく使用される評価指標です．
正解の領域と予測した領域がどれくらい重なっているかを表す指標になっています．


In [ ]:
class Evaluator(object):
    def __init__(self, num_class):
        
        self.num_class = num_class
        self.confusion_matrix = np.zeros((self.num_class,)*2)

    def Pixel_Accuracy(self):
        Acc = np.diag(self.confusion_matrix).sum() / self.confusion_matrix.sum()
        return Acc

    def Mean_Intersection_over_Union(self):
        MIoU = np.diag(self.confusion_matrix) / (
                    np.sum(self.confusion_matrix, axis=1) + np.sum(self.confusion_matrix, axis=0) -
                    np.diag(self.confusion_matrix))
        MIoU = np.nanmean(MIoU)
        return MIoU

    def _generate_matrix(self, gt_image, pre_image):
        mask = (gt_image >= 0) & (gt_image < self.num_class)
        label = self.num_class * gt_image[mask].astype('int') + pre_image[mask]
        count = np.bincount(label, minlength=self.num_class**2)
        confusion_matrix = count.reshape(self.num_class, self.num_class)
        return confusion_matrix

    def add_batch(self, gt_image, pre_image):
        assert gt_image.shape == pre_image.shape
        self.confusion_matrix += self._generate_matrix(gt_image, pre_image)

    def reset(self):
        self.confusion_matrix = np.zeros((self.num_class,) * 2)


## モデルの学習


誤差関数を設定します． 使用する誤差関数はクロスエントロピー誤差です．CrossEntropyLossをcriterionとして定義します．

学習を開始します．

各更新において，学習用データと教師データをそれぞれimageとlabelとします． 学習モデルにimageを与えて画素レベルでクラスの確率を出力するyを取得します． 各クラスの確率yと教師ラベルlabelとの誤差をcriterionで算出します． また，認識精度も算出します． そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．

segmentationの学習は時間がかかるため今回は学習済みのモデルを使用して学習します．

In [ ]:
# 学習済みモデルを呼び出す

# load_path = "./ARCdataset_png/checkpoint.pth.tar"
# checkpoint = torch.load(load_path)
# model.load_state_dict(checkpoint['state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer'])

#評価関数
evaluator = Evaluator(num_class)
# 学習の実行
loss_history=[]
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    evaluator.reset()
    # ネットワークを学習モードへ変更
    model.train()

    for sample in train_loader:

        image, label = sample['image'], sample['label']
        if use_cuda:
            image = image.cuda()
            label = label.cuda()
        y = model(image)
        loss = criterion(y, label.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #sum_loss += loss.item()

    # ネットワークを評価モードへ変更
    model.eval()
    # 評価の実行
    for sample in val_loader:
        image, label = sample['image'], sample['label']
        if use_cuda:
            image = image.cuda()
            label = label.cuda()
        with torch.no_grad():
            y = model(image)

        loss = criterion(y, label.long())
        sum_loss += loss.item()
        pred = torch.argmax(y, dim=1)
        pred = pred.data.cpu().numpy()
        label = label.cpu().numpy()
        evaluator.add_batch(label, pred) 
       
    #img_size = image.size()
    #loss_history.append(sum_loss)
    mIoU = evaluator.Mean_Intersection_over_Union()
    Acc = evaluator.Pixel_Accuracy()
    print("epoch: {}, mean loss: {}, mean accuracy: {}，　mean IoU: {}".format(epoch, sum_loss/(len(train_loader)*batch_size), Acc, mIoU))



## 学習結果
出力結果を確認しやすくするため，各クラスの色を定義する．

In [ ]:
#ラベル画像がRGB画像となっているので0～41の値に変換します
class_color = np.array([
           [  0,   0,   0],
           [ 85,   0,   0],
           [170,   0,   0],
           [255,   0,   0],
           [  0,  85,   0],
           [ 85,  85,   0],
           [170,  85,   0],
           [255,  85,   0],
           [  0, 170,   0],
           [ 85, 170,   0],
           [170, 170,   0],
           [255, 170,   0],
           [  0, 255,   0],
           [ 85, 255,   0],
           [170, 255,   0],
           [255, 255,   0],
           [  0,   0,  85],
           [ 85,   0,  85],
           [170,   0,  85],
           [255,   0,  85],
           [  0,  85,  85],
           [ 85,  85,  85],
           [170,  85,  85],
           [255,  85,  85],
           [  0, 170,  85],
           [ 85, 170,  85],
           [170, 170,  85],
           [255, 170,  85],
           [  0, 255,  85],
           [ 85, 255,  85],
           [170, 255,  85],
           [255, 255,  85],
           [  0,   0, 170],
           [ 85,   0, 170],
           [170,   0, 170],
           [255,   0, 170],
           [  0,  85, 170],
           [ 85,  85, 170],
           [170,  85, 170],
           [255,  85, 170],
           [  0, 170, 170]])



class_color = class_color[:, ::-1]
print(class_color.shape[0])

学習した結果を確認します．
入力画像，教師画像，出力画像を表示させます．\
また，クラスごとの確率マップを表示させて，各クラスの認識結果がどのようになっているか確認します．


In [ ]:
# ネットワークを評価モードへ変更
model.eval()
classes_list=['input','GT_label','output','background']
# 評価の実行
count = 0
evaluator = Evaluator(num_class)
evaluator.reset()
for sample in val_loader:
    image, label = sample['image'], sample['label']
    if use_cuda:
        image = image.cuda()
        label = label.cuda()
    with torch.no_grad():
        y = model(image)

    pred = torch.argmax(y, dim=1)
    pred = pred.data.cpu().numpy()
    label = label.cpu().numpy()
    yy = F.softmax(y,dim=1) 
    image = image.data.cpu().numpy()
    yy = yy.cpu().numpy()

    evaluator.add_batch(label, pred)        
    img_list = []

    image=image[0]
    v_img = ((image.transpose((1, 2, 0)) * [0.2023, 0.1994, 0.2010]) + [0.4914, 0.4822, 0.4465]) * 255
    v_img = np.uint8(v_img)
    img_list.append(v_img)

    result_img = np.transpose(pred, axes=[1, 2, 0])
    result_img = np.array(result_img).astype(np.uint8)
    label = np.transpose(label, axes=[1, 2, 0])
    label = np.array(label).astype(np.uint8)
    
    result_img = cv2.cvtColor(result_img, cv2.COLOR_GRAY2BGR)
    label = cv2.cvtColor(label, cv2.COLOR_GRAY2BGR)

    for i in range(0, class_color.shape[0]):
        result_img[np.where((result_img ==  [i, i, i]).all(axis=2))] = class_color[i]
        label[np.where((label ==  [i, i, i]).all(axis=2))] = class_color[i]

    label = cv2.cvtColor(label, cv2.COLOR_BGR2RGB)
    img_list.append(label)
    result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    img_list.append(result_img)

    for i in range(yy.shape[1]):
        map = yy[:,i,:,:]
        map = np.transpose(map, axes=[1, 2, 0])
        map = np.uint8(map*255)
        map = cv2.applyColorMap(map, cv2.COLORMAP_JET)
        map = cv2.cvtColor(map, cv2.COLOR_BGR2RGB)
        img_list.append(map)

    row = 6
    col = 8
    plt.figure(figsize=(18,10))
    num = 0
    while num < len(img_list):
        num += 1
        plt.subplot(row, col, num)
        plt.imshow(img_list[num-1])
        if num-1 < 4: 
            plt.title('{}'.format(classes_list[num-1]))
        else:
            plt.title('item{}'.format(num-4))  
        plt.axis('off')

    plt.show()

mIoU = evaluator.Mean_Intersection_over_Union()
Acc = evaluator.Pixel_Accuracy()
print("mean accuracy: {}, mean IoU: {}".format(Acc, mIoU))

## 学習済みモデル

数epochの学習では，まったく認識ができていないことが確認できました．\
実際．数百epochは学習しないと良い結果が出てくれません．\
今回は，400epoch学習したモデル (学習時間約4日) を読み込んで結果画像を確認します．

モデルの読み込みは`torch.load`で行います．
`model.load_state_dict`でネットワークに重みを渡します．
出力結果画像と正解画像を表示して比べます．

In [ ]:
import matplotlib.pyplot as plt
classes_list=['input','GT_label','output','background']
load_path = "./ARCdataset_png/checkpoint.pth.tar"
checkpoint = torch.load(load_path)
model.load_state_dict(checkpoint['state_dict'])


# ネットワークを評価モードへ変更
model.eval()
evaluator = Evaluator(num_class)
evaluator.reset()
# 評価の実行
count = 0
img_list = []
with torch.no_grad():
    for sample in val_loader:
        image, label = sample['image'], sample['label']

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
           
        y = model(image)
        yy = F.softmax(y,dim=1) 
        image = image.data.cpu().numpy()
        pred = y.data.cpu().numpy()
        yy = yy.data.cpu().numpy()
        label = label.cpu().numpy()
        img_list = []   

        image=image[0]
        v_img = ((image.transpose((1, 2, 0)) * [0.2023, 0.1994, 0.2010]) + [0.4914, 0.4822, 0.4465]) * 255
        v_img = np.uint8(v_img)
        img_list.append(v_img)

        pred = np.argmax(pred, axis=1)
        evaluator.add_batch(label, pred)    
        result_img = np.transpose(pred, axes=[1, 2, 0])
        result_img = np.array(result_img).astype(np.uint8)

        label = np.transpose(label, axes=[1, 2, 0])
        label = np.array(label).astype(np.uint8)
       
        result_img = cv2.cvtColor(result_img, cv2.COLOR_GRAY2BGR)
        label = cv2.cvtColor(label, cv2.COLOR_GRAY2BGR)

        for i in range(0, class_color.shape[0]):
            result_img[np.where((result_img ==  [i, i, i]).all(axis=2))] = class_color[i]
            label[np.where((label ==  [i, i, i]).all(axis=2))] = class_color[i]

        label = cv2.cvtColor(label, cv2.COLOR_BGR2RGB)
        img_list.append(label)
        result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
        img_list.append(result_img)

        for i in range(yy.shape[1]):
            map = yy[:,i,:,:]
            map = np.transpose(map, axes=[1, 2, 0])
            map = np.uint8(map*255)
            map = cv2.applyColorMap(map, cv2.COLORMAP_JET)
            map = cv2.cvtColor(map, cv2.COLOR_BGR2RGB)
            img_list.append(map)

        row = 6
        col = 8
        plt.figure(figsize=(18,10))

        num = 0

        while num < len(img_list):
            num += 1
            plt.subplot(row, col, num)
            plt.imshow(img_list[num-1])
            if num-1 < 4: 
                plt.title('{}'.format(classes_list[num-1]))
            else:
                plt.title('item{}'.format(num-4))  
            plt.axis('off')

        plt.show()

 

mIoU = evaluator.Mean_Intersection_over_Union()
Acc = evaluator.Pixel_Accuracy()
print("mean accuracy: {}，mean IoU: {}".format(Acc, mIoU))

# 課題
学習済みモデルを使って学習してみましょう

# 参考文献

V. Badrinarayanan, A. Kendall and R. Cipolla, "SegNet: A Deep Convolutional Encoder-Decoder Architecture for Image Segmentation," in IEEE Transactions on Pattern Analysis and Machine Intelligence, vol. 39, no. 12, pp. 2481-2495, 1 Dec. 2017, doi: 10.1109/TPAMI.2016.2644615.
